# Question 1: Create Notebook and transform data into pandas Dataframe (10 marks):

## Notebook created and Web page scrapped

In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests # Library for web scraping

In [3]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('BeautifulSoup  & csv imported.')

BeautifulSoup  & csv imported.


In [4]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

print('SSL certificate errors ignored.')

SSL certificate errors ignored.


In [5]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

#print(soup.prettify())
print('soup ready')

soup ready


In [6]:
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

In [7]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

## Data transformed into pandas dataframe

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 1 to 180
Data columns (total 3 columns):
PostalCode       180 non-null object
Borough          180 non-null object
Neighbourhood    180 non-null object
dtypes: object(3)
memory usage: 5.6+ KB


In [9]:
df.shape

(180, 3)

## Dataframe cleaned and notebook annotate

In [10]:
import pandas
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'lxml')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

In [11]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

In [12]:
df1 = df.reset_index()

In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
index            103 non-null int64
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


In [14]:
df1.shape

(103, 4)

In [15]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))

In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(2)
memory usage: 2.4+ KB


In [17]:
df2.shape

(103, 2)

In [18]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']

In [19]:
df3 = df2.reset_index()

In [20]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [21]:
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [22]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
PostalCode       103 non-null object
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [23]:
df3.shape

(103, 3)

# Question 2: Use the Geocoder package or the csv file to create the dataframe (2 marks)

## Used the Geocoder Package

In [24]:
!pip install geocoder

     |████████████████████████████████| 102kB 6.6MB/s ta 0:00:011


In [25]:
import pandas as pd
import numpy as np
import geocoder
print("Imported!")

Imported!


In [26]:
df3.to_csv('toronto_csv' ,index=False)

In [27]:
df3 = pd.read_csv('toronto_csv')
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [28]:
def get_latilong(postal_code):
    lati_long_coords = None
    while(lati_long_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lati_long_coords = g.latlng
    return lati_long_coords
    
get_latilong('M4G')

[43.70909000000006, -79.36409999999995]

In [29]:
postal_codes = df3['PostalCode'] 

In [30]:
coords = [ get_latilong(postal_code) for postal_code in postal_codes.tolist() ]

In [31]:
# Adding Columns Latitude & Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [32]:
df[df.PostalCode == 'M5G']

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
41,M5G,Downtown Toronto,Central Bay Street,43.68375,-79.35528


In [33]:
df.head(15)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
3,M3A,North York,Parkwoods,43.76820,-79.21761
4,M4A,North York,Victoria Village,43.76969,-79.23944
5,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.74309,-79.23526
6,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72861,-79.26367
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.71406,-79.28412
9,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.69539,-79.26194
10,M1B,Scarborough,"Malvern, Rouge",43.75998,-79.26837
12,M3B,North York,Don Mills,43.79394,-79.26711
13,M4B,East York,"Parkview Hill, Woodbine Gardens",43.78473,-79.29937
14,M5B,Downtown Toronto,"Garden District, Ryerson",43.81781,-79.28024


In [34]:
df.to_csv('toronto_part2.csv',index=False)

# Question 3:Explore and cluster the neighborhoods in Toronto (3 Marks)

In [35]:
!pip install geopy
!pip install folium
print("Installed!")

     |████████████████████████████████| 102kB 7.0MB/s ta 0:00:011
Installed!


In [36]:
import folium
import requests 
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("Imported!")

Imported!


In [37]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [38]:
import pandas as pd

import json

df.to_json(path_or_buf='geo_toronto.json', orient='table')

In [39]:
with open('geo_toronto.json') as json_data:
    Toronto_data = json.load(json_data)

In [40]:
neighborhoods_data = Toronto_data['data']
neighborhoods_data[0]
#Let's take a look at the first item in this list.

{'index': 3,
 'PostalCode': 'M3A',
 'Borough': 'North York',
 'Neighbourhood': 'Parkwoods',
 'Latitude': 43.7682,
 'Longitude': -79.21761}

In [41]:
!conda install -c conda-forge folium=0.5.0 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                       

In [42]:
import pandas as pd
import folium

print('imported pandas & folium')

imported pandas & folium


In [43]:
import folium
import requests 
import json 
import matplotlib.cm as cm
import matplotlib.colors as colors
import pandas as pd

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("Imported!")

Imported!


In [44]:
df = pd.read_csv('toronto_part2.csv')
print(df.shape)
df.head()

(103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.76820,-79.21761
1,M4A,North York,Victoria Village,43.76969,-79.23944
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.74309,-79.23526
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72861,-79.26367
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.71406,-79.28412


In [45]:
address = 'Toronto, Ontario Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of Toronto Canada are 43.6534817, -79.3839347.


In [46]:
toronto_data = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(39, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.74309,-79.23526
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.71406,-79.28412
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.81781,-79.28024
3,M5C,Downtown Toronto,St. James Town,43.74787,-79.40029
4,M4E,East Toronto,The Beaches,43.74107,-79.51080


In [57]:
# Foursquare API
CLIENT_ID = 'MSCJGUZIPVEJAEGS2NF3KNL12GSOMVJUJOEGQLIQHATWA55X'
CLIENT_SECRET = 'GTRHCKYH4QGMIQQEV2RKTPH0CHUBN0AIURJJ2EBD11GHEDLA' 
VERSION = '20200713'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: Hidden')
print('CLIENT_SECRET: Hidden')

Your credentails:
CLIENT_ID: Hidden
CLIENT_SECRET: Hidden


In [75]:
neighbourhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.743090000000045, -79.23525999999998.


In [77]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=MSCJGUZIPVEJAEGS2NF3KNL12GSOMVJUJOEGQLIQHATWA55X&client_secret=GTRHCKYH4QGMIQQEV2RKTPH0CHUBN0AIURJJ2EBD11GHEDLA&v=20200713&ll=43.743090000000045,-79.23525999999998&radius=500&limit=100'

In [78]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0c9d8a461da410d64daa42'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Eglinton East',
  'headerFullLocation': 'Eglinton East, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 43.74759000450005,
    'lng': -79.22904279614892},
   'sw': {'lat': 43.73858999550004, 'lng': -79.24147720385105}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb3e9c2f187a593fea213f8',
       'name': 'Dairy Queen',
       'location': {'address': '2916 Eglinton Ave E',
        'lat': 43.73950592131142,
        'lng': -79.23689366388604,
        'labeledLatLngs': [{'label': 'display',
          'lat'

In [79]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [89]:
import json
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Dairy Queen,Ice Cream Shop,43.739506,-79.236894
1,Dairy Queen,Ice Cream Shop,43.739580,-79.236991
2,Swiss Chalet,Restaurant,43.740393,-79.232802
3,Eglinton GO Station,Train Station,43.739701,-79.232281


In [90]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [91]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North


KeyError: 'groups'

In [92]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North


KeyError: 'groups'

In [93]:
print(toronto_venues.shape)
toronto_venues.head()

NameError: name 'toronto_venues' is not defined